In [30]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from transformers import BertTokenizer, BertModel
import spacy






In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [32]:
# Load dataset
file_path = 'full_format_recipes.json' 
data = pd.read_json(file_path)




In [ ]:

# Basic exploration
print(f"Dataset shape: {data.shape}")
print("Columns:", data.columns)

In [ ]:
# Focus on relevant columns
FilteredData1 = data[['directions', 'desc', 'rating']].copy()
FilteredData1 = FilteredData1.dropna()



In [ ]:
# Analyze relationships ##TODO: SMTH better than this shit
print(FilteredData1.groupby('rating').size())


In [36]:
# Text preprocessing function
nlp = spacy.load('en_core_web_sm')
def preprocess_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])


In [ ]:


# Apply preprocessing
FilteredData1['processed_directions'] = FilteredData1['directions'].apply(lambda x: preprocess_text(" ".join(x) if isinstance(x, list) else x))
FilteredData1['processed_desc'] = FilteredData1['desc'].apply(lambda x: preprocess_text(" ".join(x) if isinstance(x, list) else x))

# Display a few samples of the processed data
print(FilteredData1[['processed_directions', 'processed_desc']].head())


In [9]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    FilteredData1['processed_directions'], FilteredData1['rating'], test_size=0.2, random_state=42)

In [10]:
# TF-IDF Representation
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [ ]:
# Model pipeline for Random Forest
rf_pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)



In [ ]:

# Predictions and evaluation
rf_predictions = rf_pipeline.predict(X_test)
print("Random Forest MSE:", mean_squared_error(y_test, rf_predictions))
print("Random Forest R2 Score:", r2_score(y_test, rf_predictions))

In [22]:
# Using BERT embeddings for contextual embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [23]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()


In [24]:

# Generate BERT embeddings for a small subset (for demonstration purposes)
#FilteredData1_reduced100 = FilteredData1[:100]  # Use smaller subset to avoid memory issues
sample_embeddings = np.vstack(FilteredData1['processed_directions'].apply(get_bert_embedding))

In [25]:
# Split embeddings
y_sample = FilteredData1['rating']
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
    sample_embeddings, y_sample, test_size=0.2, random_state=42)



In [26]:
# Train a neural network using PyTorch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.fc(x)

input_size = sample_embeddings.shape[1]
model = SimpleNN(input_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train_bert, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_bert.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test_bert, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_bert.values, dtype=torch.float32).view(-1, 1).to(device)

In [ ]:
# Training loop
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")



In [ ]:
# Evaluation

model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print("Neural Network Test Loss:", test_loss.item())
